In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### **Step 1 : 建模**
##### M1 = 9447 ~ 9477 / M2 = 9477 ~ 9507 / M3 = 9507 ~ 9537 / M4 = 9537 ~ 9567

In [3]:
tb_data = './data/'

#### **1. 匯入所有商品30~60(m3)有無購買資料, 基本屬性資料, 網址資料**

In [4]:
fx_m3 = pd.read_csv('./data/fx_m3.csv')
cc_m3 = pd.read_csv('./data/cc_m3.csv')
ln_m3 = pd.read_csv('./data/ln_m3.csv')
wm_m3 = pd.read_csv('./data/wm_m3.csv')
cif_use = pd.read_csv('./data/cif_use.csv')
web_train = pd.read_csv('./data/web_train.csv')
# cif_use.drop(['CUST_START_DT'], axis = 1, inplace = True)

In [5]:
cif_use.loc[~cif_use['GENDER_CODE'].isnull(), :].shape

(187679, 7)

#### **2. 處理外匯資料**

In [6]:
FX = os.path.join(tb_data, 'TBN_FX_TXN.csv')
tb_fx = pd.read_csv(FX)
tb_fx.rename(columns={'TXN_DT':'FX_DT'}, inplace = True)
# tb_fx.sort_index(inplace = True)

In [7]:
# 計算30天以上,120天以內購買次數(m1~m3, 9447~9537)
fx_90 = tb_fx.loc[tb_fx['FX_DT'] <= 9537,['CUST_NO', 'FX_DT']]
fx_90.reset_index(drop=True, inplace=True)
fx_90['FX_DT'] = 1
fx_90 = fx_90.groupby('CUST_NO').sum().reset_index(level=0)
fx_90.columns = ['CUST_NO','FX_count']

In [20]:
# 計算30天以上, 120天以內購買金額總數
fx_90_amt = tb_fx.loc[tb_fx['FX_DT'] <= 9537,['CUST_NO', 'FX_TXN_AMT']]
fx_90_amt.reset_index(drop=True, inplace=True)
fx_90_amt = fx_90_amt.groupby('CUST_NO').sum().reset_index(level=0)
fx_90_amt.columns = ['CUST_NO','AMT_SUM']
#標記購買金額區間
fx_90_amt['AMT_RANGE'] = pd.qcut(fx_90_amt['AMT_SUM'], 4, labels = [1, 2, 3, 4]) 
#併入fx_90
fx_90['AMT_RANGE'] = fx_90_amt['AMT_RANGE'] 

In [9]:
fx_90['AMT_RANGE'].value_counts()

1    17050
4    17049
3    17049
2    17049
Name: AMT_RANGE, dtype: int64

In [22]:
# 篩出0~30天(m4)有無購買(Y)
tb_fx['FX_Y'] = [1 if i >= 9537 else 0 for i in tb_fx['FX_DT']]
fx_y = tb_fx.loc[tb_fx['FX_Y'] == 1,['CUST_NO', 'FX_Y']].drop_duplicates(subset = 'CUST_NO')

In [23]:
# 合併fx所需資料
fx_train = fx_90.merge(fx_y, on = 'CUST_NO', how = 'left')
fx_train['FX_Y'] = fx_train['FX_Y'].fillna(0)

In [24]:
fx_train.isnull().any()

CUST_NO      False
FX_count     False
AMT_RANGE    False
FX_Y         False
dtype: bool

In [30]:
#取出購買金額區間之dummy，合併cust_no，並計算單一客戶購買金額區間之次數
tb_fx_dummy = pd.get_dummies(tb_fx_30['AMT_RANGE'])
tb_fx_m = tb_fx_30[['CUST_NO']].join(tb_fx_dummy)
tb_fx_m = tb_fx_m.groupby(['CUST_NO']).sum().reset_index(level=0)

In [118]:
# 計算客戶購買次數
# fx_count = tb_fx['CUST_NO'].value_counts().to_frame()
# fx_count = fx_count.to_frame()
# fx_count.reset_index(inplace = True)
# fx_count.columns = ['CUST_NO','fx_count']
# fx_count.shape

#### **3. 以基本屬性為底,合併網址、外匯、其他商品m3有無購買之資料**

In [25]:
fx_forecast = cif_use.merge(web_train, on = 'CUST_NO', how = 'left').merge(fx_train, on = 'CUST_NO', how = 'left').merge(fx_m3, on = 'CUST_NO', how = 'left')\
.merge(ln_m3, on = 'CUST_NO', how = 'left').merge(wm_m3, on = 'CUST_NO', how = 'left').merge(cc_m3, on = 'CUST_NO', how = 'left')

In [26]:
fx_forecast.loc[~fx_forecast['GENDER_CODE'].isnull(),:].shape

(187679, 21)

In [29]:
fx_forecast.isnull().any()

CUST_NO              False
AGE                  False
CHILDREN_CNT         False
EDU_CODE             False
GENDER_CODE          False
INCOME_RANGE_CODE    False
WORK_MTHS            False
web_1                False
web_2                False
web_3                False
web_4                False
web_5                False
web_6                False
web_7                False
FX_count             False
AMT_RANGE            False
FX_Y                 False
FX_M3                False
LN_M3                False
WM_M3                False
CC_M3                False
dtype: bool

In [28]:
fx_forecast_fill = [i for i in fx_forecast.columns if i !='CUST_NO' and i !='GENDER_CODE' and i != 'AMT_RANGE'] 
fx_forecast[fx_forecast_fill] = fx_forecast[fx_forecast_fill].fillna(0)
fx_forecast['AMT_RANGE'] = fx_forecast['AMT_RANGE'].cat.add_categories('0')
fx_forecast['AMT_RANGE'].fillna('0', inplace =True)
# fx_forecast['GENDER_CODE'] = fx_forecast['GENDER_CODE'].fillna(2)
fx_forecast_list = [i for i in fx_forecast.columns if i !='CUST_NO' and i != 'AMT_RANGE'] 
fx_forecast[fx_forecast_list] = fx_forecast[fx_forecast_list].apply(lambda x:x.astype(int))

#### **4. 決策樹**

In [30]:
X = fx_forecast.drop(['CUST_NO', 'FX_Y'], axis = 1)
Y = fx_forecast['FX_Y']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30,random_state=0)

In [31]:
Y.value_counts()

0    176957
1     10722
Name: FX_Y, dtype: int64

In [32]:
print('X_train :',X_train.shape)
print('Y_train', Y_train.shape)
print('X_test :', X_test.shape)
print('Y_test', Y_test.shape)

X_train : (131375, 19)
Y_train (131375,)
X_test : (56304, 19)
Y_test (56304,)


In [33]:
fx_tree = DecisionTreeClassifier(criterion = "entropy",
                               max_depth=3, min_samples_leaf=5)
fx_tree.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [34]:
fx_predict = fx_tree.predict(X_test)

In [35]:
# pd.Series(fx_predict.tolist()).value_counts()

In [36]:
# print(classification_report(Y_test,fx_predict))
print("Accuracy is ", accuracy_score(Y_test, fx_predict)*100)
# print(confusion_matrix(y_test,predictions))

Accuracy is  95.00035521454959


In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

##### **4.1 跑個圖**

In [24]:
X = fx_forecast.drop(['CUST_NO', 'FX_Y'], axis = 1)
Y = fx_forecast['FX_Y']
fx = tree.DecisionTreeClassifier()
fx = fx.fit(X,Y)

In [25]:
X.columns

Index(['AGE', 'CHILDREN_CNT', 'EDU_CODE', 'GENDER_CODE', 'INCOME_RANGE_CODE',
       'WORK_MTHS', 'web_1', 'web_2', 'web_3', 'web_4', 'web_5', 'web_6',
       'web_7', 'FX_count', 'AMT_RANGE', 'FX_M3', 'LN_M3', 'WM_M3', 'CC_M3'],
      dtype='object')

In [29]:
import os
os.environ['PATH'] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [31]:
# import graphviz 
# fx_data = tree.export_graphviz(fx, out_file=None,
#                                feature_names=X.columns,  
#                                class_names='FX_Y',  
#                                filled=True, rounded=True,  
#                                special_characters=True)  
# graph = graphviz.Source(fx_data)  
# graph 

### **Step 2 : Load 預測資料集**

In [37]:
Y_zero  = pd.read_csv('./data/TBN_Y_ZERO.csv')
# Y_zero  = pd.read_csv('./data/TBN_Y_ZERO.csv', usecols = ['CUST_NO', 'FX_IND'])

#### **1. 匯入所有商品0~30(m4)有無購買,網址資料(m3,m4)**

In [38]:
fx_m4 = pd.read_csv('./data/fx_m4.csv')
cc_m4 = pd.read_csv('./data/cc_m4.csv')
ln_m4 = pd.read_csv('./data/ln_m4.csv')
wm_m4 = pd.read_csv('./data/wm_m4.csv')
web_test = pd.read_csv('./data/web_test.csv')

In [39]:
web_test.shape

(106655, 8)

#### **2. 處理外匯資料**

In [40]:
# 計算0~90天以內購買次數(m2~m4, 9477~9567)
fx_24 = tb_fx.loc[tb_fx['FX_DT'] > 9477,['CUST_NO', 'FX_DT']]
fx_24.reset_index(drop=True, inplace=True)
fx_24['FX_DT'] = 1
fx_24 = fx_24.groupby('CUST_NO').sum().reset_index(level=0)
fx_24.columns = ['CUST_NO','FX_count']

In [44]:
# # 計算0~90天以內購買金額平均
# fx_24_amt = tb_fx.loc[tb_fx['FX_DT'] > 9477,['CUST_NO', 'FX_TXN_AMT']]
# fx_24_amt.reset_index(drop=True, inplace=True)
# fx_24_amt = fx_24_amt.groupby('CUST_NO').mean().reset_index(level=0)
# fx_24_amt.columns = ['CUST_NO','AMT_AVG']
# 計算0~90天以內購買金額總數
fx_24_amt = tb_fx.loc[tb_fx['FX_DT'] > 9477,['CUST_NO', 'FX_TXN_AMT']]
fx_24_amt.reset_index(drop=True, inplace=True)
fx_24_amt = fx_24_amt.groupby('CUST_NO').sum().reset_index(level=0)
fx_24_amt.columns = ['CUST_NO','AMT_SUM']
#標記購買金額區間
fx_24_amt['AMT_RANGE'] = pd.qcut(fx_24_amt['AMT_SUM'], 4, labels = [1, 2, 3, 4]) 
#併入fx_24
fx_24['AMT_RANGE'] = fx_24_amt['AMT_RANGE']

In [45]:
# 合併fx所需資料
fx_test = fx_24
fx_test.isnull().any()

CUST_NO      False
FX_count     False
AMT_RANGE    False
dtype: bool

#### **3. 以Y為底，合併基本屬性,網址、外匯、其他商品m3有無購買之資料**

In [80]:
Y_fx = Y_zero[['CUST_NO']].merge(cif_use, on = 'CUST_NO', how = 'left').merge(web_test, on = 'CUST_NO', how = 'left').merge(fx_test, on = 'CUST_NO', how = 'left')\
.merge(fx_m4, on = 'CUST_NO', how = 'left').merge(wm_m4, on = 'CUST_NO', how = 'left').merge(cc_m4, on = 'CUST_NO', how = 'left').merge(ln_m4, on = 'CUST_NO', how = 'left')

In [61]:
Y_fx.loc[~Y_fx['AGE'].isnull(),:].shape

(29854, 20)

In [85]:
Y_fx.isnull().any()
# Y_fx.info()

CUST_NO              False
AGE                  False
CHILDREN_CNT         False
EDU_CODE             False
GENDER_CODE          False
INCOME_RANGE_CODE    False
WORK_MTHS            False
web_1                False
web_2                False
web_3                False
web_4                False
web_5                False
web_6                False
web_7                False
FX_count             False
AMT_RANGE            False
FX_M4                False
WM_M4                False
CC_M4                False
LN_M4                False
dtype: bool

##### **3.1 處理基本屬性資料**

In [82]:
m_edu = round(Y_fx.loc[:,'EDU_CODE'].mean())
m_income = round(Y_fx.loc[:, 'INCOME_RANGE_CODE'].mean())
m_work = round(Y_fx.loc[:, 'WORK_MTHS'].mean())
m_age = round(Y_fx.loc[:, 'AGE'].mean())
name_ = Y_fx.columns.tolist()[1:7]
mean_ = [m_age, 0, m_edu, 2, m_income, m_work] 
for i,j in zip(name_, mean_):
    Y_fx.loc[:,i] = Y_fx.loc[:, i].fillna(j)
# Y_fx.loc[:,'CHILDREN_CNT'] = Y_fx.loc[:,'CHILDREN_CNT'].fillna(0) 
# Y_fx.loc[:,'EDU_CODE'] = Y_fx.loc[:,'EDU_CODE'].fillna(m_edu)
# Y_fx.loc[:,'INCOME_RANGE_CODE'] = Y_fx.loc[:,'INCOME_RANGE_CODE'].fillna(m_income)
# Y_fx.loc[:,'WORK_MTHS'] = Y_fx.loc[:,'WORK_MTHS'].fillna(m_work)
# Y_fx.loc[:,'AGE'] = Y_fx.loc[:,'WORK_MTHS'].fillna(m_age)
# Y_fx['GENDER_CODE'] = Y_fx['GENDER_CODE'].fillna(2)

##### 3.2 **處理剩餘na**

In [ ]:
fx_forecast['AMT_RANGE'] = fx_forecast['AMT_RANGE'].cat.add_categories('0')
fx_forecast['AMT_RANGE'].fillna('0', inplace =True)
# fx_forecast['GENDER_CODE'] = fx_forecast['GENDER_CODE'].fillna(2)
fx_forecast_list = [i for i in fx_forecast.columns if i !='CUST_NO' and i != 'AMT_RANGE'] 
fx_forecast[fx_forecast_list] = fx_forecast[fx_forecast_list].apply(lambda x:x.astype(int))

In [83]:
Y_fx['AMT_RANGE'] = Y_fx['AMT_RANGE'].cat.add_categories('0')
Y_fx['AMT_RANGE'].fillna('0', inplace = True)

In [84]:
# Y_fx = Y_fx.fillna(0)
Y_fx_list = [i for i in Y_fx.columns if i !='CUST_NO' and i != 'AMT_RANGE']
Y_fx[Y_fx_list] = Y_fx[Y_fx_list].fillna(0)
Y_fx[Y_fx_list] = Y_fx[Y_fx_list].apply(lambda x:x.astype(int))

#### **4. 決策樹**

In [86]:
X_train = fx_forecast.drop(['CUST_NO', 'FX_Y'], axis = 1)
Y_train = fx_forecast['FX_Y']
X_test = Y_fx.drop(['CUST_NO'], axis = 1)

In [87]:
print('X_train :',X_train.shape)
print('Y_train', Y_train.shape)
print('X_test :', X_test.shape)
# print('Y_test', Y_test.shape)

X_train : (187679, 19)
Y_train (187679,)
X_test : (30000, 19)


In [88]:
fx_tree = DecisionTreeClassifier(criterion = "entropy",
                               max_depth=3, min_samples_leaf=5)
fx_tree.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [89]:
fx_predict = fx_tree.predict(X_test).tolist()

In [90]:
Y_zero['FX_IND'] = fx_predict

In [95]:
# Y_zero[Y_zero['FX_IND'] == 1]

In [98]:
TBN_Y_FX = Y_zero.loc[:,['CUST_NO', 'FX_IND']]
TBN_Y_FX.head()
TBN_Y_FX.to_csv('./data/TBN_Y_FX.csv', index = False)

### **Extra : Random Forest**